In [1]:

import os
import pandas as pd
from tabulate import tabulate
from dotenv import load_dotenv
import snowflake.connector
import sys
from datetime import datetime, timedelta
from tqdm import tqdm

load_dotenv()

# Create data folder if it doesn't exist
os.makedirs('data', exist_ok=True)

# --- Snowflake connection ---
conn = snowflake.connector.connect(
    user=os.getenv('SNOWFLAKE_USER'),
    password=os.getenv('SNOWFLAKE_PASSWORD'),
    account=os.getenv('SNOWFLAKE_ACCOUNT'),
    warehouse=os.getenv('SNOWFLAKE_WAREHOUSE', 'COMPUTE_WH'),
    database='INCREMENTALITY',
    schema='INCREMENTALITY_RESEARCH'
)
cursor = conn.cursor()

def run_query(query):
    try:
        cursor.execute(query)
        if cursor.description:
            results = cursor.fetchall()
            columns = [desc[0] for desc in cursor.description]
            return pd.DataFrame(results, columns=columns)
        return pd.DataFrame()
    except snowflake.connector.ProgrammingError as e:
        print(f"\nERROR executing query:\n{query}\nDetails: {e}")
        raise

def show_table(df, title=""):
    if title:
        print(f"\n{title}")
        print("="*len(title))
    print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))

print("✅ Connected to Snowflake")


✅ Connected to Snowflake


# purchases

In [5]:

  # --- CONFIGURATION ---
  ANALYSIS_START_DATE = '2025-03-01'
  ANALYSIS_END_DATE = '2025-09-30'

  print(f"\n--- Collecting Half-Hourly Raw Purchase Metrics (Weekly Pulls) ---")
  print(f"Period: {ANALYSIS_START_DATE} to {ANALYSIS_END_DATE}")

  # --- Main Loop ---
  all_half_hourly_data = []
  current_week_start = datetime.strptime(ANALYSIS_START_DATE, '%Y-%m-%d')
  end_date_obj = datetime.strptime(ANALYSIS_END_DATE, '%Y-%m-%d')

  week_count = (end_date_obj - current_week_start).days // 7 + 1
  progress_bar = tqdm(total=week_count, desc="Processing weeks")

  while current_week_start < end_date_obj:
      current_week_end = current_week_start + timedelta(days=7)

      week_start_str = current_week_start.strftime('%Y-%m-%d 00:00:00')
      week_end_str = current_week_end.strftime('%Y-%m-%d 00:00:00')

      print(f"Processing week: {current_week_start.date()} to {current_week_end.date()}...")

      query = f"""
      SELECT
          -- Half-hourly time bucket
          DATEADD(MINUTE, FLOOR(EXTRACT(MINUTE FROM PURCHASED_AT)/30)*30, 
                  DATE_TRUNC('HOUR', PURCHASED_AT))::TIMESTAMP_NTZ AS activity_half_hour,

          -- Raw Aggregate Metrics
          COUNT(DISTINCT PURCHASE_ID) AS half_hourly_transaction_count,
          COALESCE(SUM(QUANTITY * UNIT_PRICE), 0) AS half_hourly_gmv,
          COALESCE(SUM(QUANTITY), 0) AS half_hourly_units_sold,
          COUNT(DISTINCT USER_ID) AS half_hourly_purchasing_users,
          COUNT(DISTINCT PRODUCT_ID) AS half_hourly_products_purchased,
          COUNT(*) AS half_hourly_total_purchase_lines,
          
          -- Price and transaction metrics
          COALESCE(AVG(QUANTITY * UNIT_PRICE), 0) AS half_hourly_avg_transaction_value,
          COALESCE(AVG(QUANTITY), 0) AS half_hourly_avg_units_per_line,
          COALESCE(AVG(UNIT_PRICE), 0) AS half_hourly_avg_unit_price,
          COALESCE(MIN(UNIT_PRICE), 0) AS half_hourly_min_unit_price,
          COALESCE(MAX(UNIT_PRICE), 0) AS half_hourly_max_unit_price,
          COALESCE(STDDEV(UNIT_PRICE), 0) AS half_hourly_stddev_unit_price,
          
          -- Quantity metrics
          COALESCE(MIN(QUANTITY), 0) AS half_hourly_min_quantity,
          COALESCE(MAX(QUANTITY), 0) AS half_hourly_max_quantity,
          COALESCE(STDDEV(QUANTITY), 0) AS half_hourly_stddev_quantity
      FROM
          PURCHASES
      WHERE
          PURCHASED_AT >= '{week_start_str}'::TIMESTAMP_NTZ
          AND PURCHASED_AT < '{week_end_str}'::TIMESTAMP_NTZ
      GROUP BY
          1
      ORDER BY
          1;
      """

      try:
          half_hourly_df_for_week = run_query(query)
          if not half_hourly_df_for_week.empty:
              all_half_hourly_data.append(half_hourly_df_for_week)
              print(f"   -> Success: Found {half_hourly_df_for_week.shape[0]} half-hourly records for the week.")
          else:
              print(f"   -> Info: No purchase data found for this week.")

      except Exception as e:
          print(f"   -> ERROR processing week starting {current_week_start.date()}: {e}")

      # Move to the next week
      current_week_start = current_week_end
      progress_bar.update(1)

  progress_bar.close()

  # --- Final Processing and Display ---
  if all_half_hourly_data:
      final_df = pd.concat(all_half_hourly_data, ignore_index=True)

      # Convert all numeric columns
      numeric_columns = [col for col in final_df.columns if col != 'ACTIVITY_HALF_HOUR']
      for col in numeric_columns:
          final_df[col] = pd.to_numeric(final_df[col])

      # --- SAVE TO PARQUET IN DATA FOLDER ---
      output_filename = f"data/half_hourly_purchases_{ANALYSIS_START_DATE}_to_{ANALYSIS_END_DATE}.parquet"
      final_df.to_parquet(output_filename, index=False)
      print(f"\n✅ Data successfully saved to {output_filename}")

      show_table(final_df.head(25), "Aggregated Half-Hourly Purchase Metrics")

      print(f"\nDataset Summary:")
      print(f"Total records: {final_df.shape[0]:,}")
      print(f"Total columns: {final_df.shape[1]}")
      print(f"Memory usage: {final_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
  else:
      print("\nNo purchase data found for the entire specified period.")


--- Collecting Half-Hourly Raw Purchase Metrics (Weekly Pulls) ---
Period: 2025-03-01 to 2025-09-30


Processing weeks:  10%|▉         | 3/31 [03:41<34:31, 73.97s/it]

Processing week: 2025-03-01 to 2025-03-08...



Processing weeks:   3%|▎         | 1/31 [00:02<01:21,  2.71s/it]

   -> Info: No purchase data found for this week.
Processing week: 2025-03-08 to 2025-03-15...


...

   -> Success: Found 336 half-hourly records for the week.
Processing week: 2025-09-13 to 2025-09-20...


Processing weeks:  97%|█████████▋| 30/31 [02:21<00:04,  4.66s/it]

   -> Success: Found 191 half-hourly records for the week.
Processing week: 2025-09-20 to 2025-09-27...
   -> Info: No purchase data found for this week.
Processing week: 2025-09-27 to 2025-10-04...


Processing weeks: 100%|██████████| 31/31 [02:21<00:00,  4.58s/it]

   -> Info: No purchase data found for this week.

✅ Data successfully saved to data/half_hourly_purchases_2025-03-01_to_2025-09-30.parquet

Aggregated Half-Hourly Purchase Metrics
+----------------------+---------------------------------+-------------------+--------------------------+--------------------------------+----------------------------------+------------------------------------+-------------------------------------+----------------------------------+------------------------------+------------------------------+------------------------------+---------------------------------+----------------------------+----------------------------+-------------------------------+
| ACTIVITY_HALF_HOUR   |   HALF_HOURLY_TRANSACTION_COUNT |   HALF_HOURLY_GMV |   HALF_HOURLY_UNITS_SOLD |   HALF_HOURLY_PURCHASING_USERS |   HALF_HOURLY_PRODUCTS_PURCHASED |   HALF_HOURLY_TOTAL_PURCHASE_LINES |   HALF_HOURLY_AVG_TRANSACTION_VALUE |   HALF_HOURLY_AVG_UNITS_PER_LINE |   HALF_HOURLY_AVG_UNIT_PRICE |   H

# clicks

In [6]:

# --- CONFIGURATION ---
ANALYSIS_START_DATE = '2025-03-01'
ANALYSIS_END_DATE = '2025-09-30'

print(f"\n--- Collecting Half-Hourly Raw Click Metrics (Weekly Pulls) ---")
print(f"Period: {ANALYSIS_START_DATE} to {ANALYSIS_END_DATE}")

# --- Main Loop ---
all_half_hourly_data = []
current_week_start = datetime.strptime(ANALYSIS_START_DATE, '%Y-%m-%d')
end_date_obj = datetime.strptime(ANALYSIS_END_DATE, '%Y-%m-%d')

week_count = (end_date_obj - current_week_start).days // 7 + 1
progress_bar = tqdm(total=week_count, desc="Processing weeks")

while current_week_start < end_date_obj:
    current_week_end = current_week_start + timedelta(days=7)

    week_start_str = current_week_start.strftime('%Y-%m-%d 00:00:00')
    week_end_str = current_week_end.strftime('%Y-%m-%d 00:00:00')

    print(f"Processing week: {current_week_start.date()} to {current_week_end.date()}...")

    query = f"""
    SELECT
        -- Half-hourly time bucket
        DATEADD(MINUTE, FLOOR(EXTRACT(MINUTE FROM OCCURRED_AT)/30)*30, 
                DATE_TRUNC('HOUR', OCCURRED_AT))::TIMESTAMP_NTZ AS activity_half_hour,

        -- Raw Aggregate Metrics
        COUNT(INTERACTION_ID) AS half_hourly_click_count,
        COUNT(DISTINCT USER_ID) AS half_hourly_clicking_users,
        COUNT(DISTINCT VENDOR_ID) AS half_hourly_clicked_vendors,
        COUNT(DISTINCT CAMPAIGN_ID) AS half_hourly_clicked_campaigns,
        COUNT(DISTINCT PRODUCT_ID) AS half_hourly_clicked_products,
        COUNT(DISTINCT AUCTION_ID) AS half_hourly_clicked_auctions,
        
        -- Engagement intensity metrics
        COUNT(DISTINCT CONCAT(USER_ID, '_', VENDOR_ID)) AS half_hourly_user_vendor_pairs,
        COUNT(DISTINCT CONCAT(USER_ID, '_', PRODUCT_ID)) AS half_hourly_user_product_pairs,
        COUNT(DISTINCT CONCAT(VENDOR_ID, '_', PRODUCT_ID)) AS half_hourly_vendor_product_pairs,
        COUNT(DISTINCT CONCAT(CAMPAIGN_ID, '_', PRODUCT_ID)) AS half_hourly_campaign_product_pairs,
        COUNT(DISTINCT CONCAT(USER_ID, '_', CAMPAIGN_ID)) AS half_hourly_user_campaign_pairs
    FROM
        CLICKS
    WHERE
        OCCURRED_AT >= '{week_start_str}'::TIMESTAMP_NTZ
        AND OCCURRED_AT < '{week_end_str}'::TIMESTAMP_NTZ
    GROUP BY
        1
    ORDER BY
        1;
    """

    try:
        half_hourly_df_for_week = run_query(query)
        if not half_hourly_df_for_week.empty:
            all_half_hourly_data.append(half_hourly_df_for_week)
            print(f"   -> Success: Found {half_hourly_df_for_week.shape[0]} half-hourly records for the week.")
        else:
            print(f"   -> Info: No click data found for this week.")

    except Exception as e:
        print(f"   -> ERROR processing week starting {current_week_start.date()}: {e}")

    # Move to the next week
    current_week_start = current_week_end
    progress_bar.update(1)

progress_bar.close()

# --- Final Processing and Display ---
if all_half_hourly_data:
    final_df = pd.concat(all_half_hourly_data, ignore_index=True)

    # Convert all numeric columns
    numeric_columns = [col for col in final_df.columns if col != 'ACTIVITY_HALF_HOUR']
    for col in numeric_columns:
        final_df[col] = pd.to_numeric(final_df[col])

    # --- SAVE TO PARQUET IN DATA FOLDER ---
    output_filename = f"data/half_hourly_clicks_{ANALYSIS_START_DATE}_to_{ANALYSIS_END_DATE}.parquet"
    final_df.to_parquet(output_filename, index=False)
    print(f"\n✅ Data successfully saved to {output_filename}")

    show_table(final_df.head(25), "Aggregated Half-Hourly Click Metrics")

    print(f"\nDataset Summary:")
    print(f"Total records: {final_df.shape[0]:,}")
    print(f"Total columns: {final_df.shape[1]}")
    print(f"Memory usage: {final_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
else:
    print("\nNo click data found for the entire specified period.")



--- Collecting Half-Hourly Raw Click Metrics (Weekly Pulls) ---
Period: 2025-03-01 to 2025-09-30


Processing weeks:   0%|          | 0/31 [00:00<?, ?it/s]

Processing week: 2025-03-01 to 2025-03-08...


Processing weeks:   3%|▎         | 1/31 [00:03<01:43,  3.47s/it]

   -> Info: No click data found for this week.
Processing week: 2025-03-08 to 2025-03-15...


...

   -> Success: Found 191 half-hourly records for the week.
Processing week: 2025-09-20 to 2025-09-27...


Processing weeks:  97%|█████████▋| 30/31 [14:28<00:23, 23.41s/it]

   -> Info: No click data found for this week.
Processing week: 2025-09-27 to 2025-10-04...


Processing weeks: 100%|██████████| 31/31 [14:29<00:00, 28.04s/it]

   -> Info: No click data found for this week.

✅ Data successfully saved to data/half_hourly_clicks_2025-03-01_to_2025-09-30.parquet

Aggregated Half-Hourly Click Metrics
+----------------------+---------------------------+------------------------------+-------------------------------+---------------------------------+--------------------------------+--------------------------------+---------------------------------+----------------------------------+------------------------------------+--------------------------------------+-----------------------------------+
| ACTIVITY_HALF_HOUR   |   HALF_HOURLY_CLICK_COUNT |   HALF_HOURLY_CLICKING_USERS |   HALF_HOURLY_CLICKED_VENDORS |   HALF_HOURLY_CLICKED_CAMPAIGNS |   HALF_HOURLY_CLICKED_PRODUCTS |   HALF_HOURLY_CLICKED_AUCTIONS |   HALF_HOURLY_USER_VENDOR_PAIRS |   HALF_HOURLY_USER_PRODUCT_PAIRS |   HALF_HOURLY_VENDOR_PRODUCT_PAIRS |   HALF_HOURLY_CAMPAIGN_PRODUCT_PAIRS |   HALF_HOURLY_USER_CAMPAIGN_PAIRS |
+======================+==========

# impressions

In [5]:

# --- CONFIGURATION ---
ANALYSIS_START_DATE = '2025-03-01'
ANALYSIS_END_DATE = '2025-09-30'

print(f"\n--- Collecting Half-Hourly Raw Impression Metrics (Weekly Pulls) ---")
print(f"Period: {ANALYSIS_START_DATE} to {ANALYSIS_END_DATE}")

# --- Main Loop ---
all_half_hourly_data = []
current_week_start = datetime.strptime(ANALYSIS_START_DATE, '%Y-%m-%d')
end_date_obj = datetime.strptime(ANALYSIS_END_DATE, '%Y-%m-%d')

week_count = (end_date_obj - current_week_start).days // 7 + 1
progress_bar = tqdm(total=week_count, desc="Processing weeks")

while current_week_start < end_date_obj:
    current_week_end = current_week_start + timedelta(days=7)

    week_start_str = current_week_start.strftime('%Y-%m-%d 00:00:00')
    week_end_str = current_week_end.strftime('%Y-%m-%d 00:00:00')

    print(f"Processing week: {current_week_start.date()} to {current_week_end.date()}...")

    query = f"""
    SELECT
        -- Half-hourly time bucket
        DATEADD(MINUTE, FLOOR(EXTRACT(MINUTE FROM OCCURRED_AT)/30)*30, 
                DATE_TRUNC('HOUR', OCCURRED_AT))::TIMESTAMP_NTZ AS activity_half_hour,

        -- Raw Aggregate Metrics
        COUNT(INTERACTION_ID) AS half_hourly_impression_count,
        COUNT(DISTINCT USER_ID) AS half_hourly_impressed_users,
        COUNT(DISTINCT VENDOR_ID) AS half_hourly_impressed_vendors,
        COUNT(DISTINCT CAMPAIGN_ID) AS half_hourly_impressed_campaigns,
        COUNT(DISTINCT PRODUCT_ID) AS half_hourly_impressed_products,
        COUNT(DISTINCT AUCTION_ID) AS half_hourly_impressed_auctions,
        
        -- Concentration metrics
        COUNT(DISTINCT CONCAT(USER_ID, '_', VENDOR_ID)) AS half_hourly_user_vendor_pairs,
        COUNT(DISTINCT CONCAT(USER_ID, '_', PRODUCT_ID)) AS half_hourly_user_product_pairs,
        COUNT(DISTINCT CONCAT(VENDOR_ID, '_', PRODUCT_ID)) AS half_hourly_vendor_product_pairs,
        COUNT(DISTINCT CONCAT(CAMPAIGN_ID, '_', PRODUCT_ID)) AS half_hourly_campaign_product_pairs
    FROM
        IMPRESSIONS
    WHERE
        OCCURRED_AT >= '{week_start_str}'::TIMESTAMP_NTZ
        AND OCCURRED_AT < '{week_end_str}'::TIMESTAMP_NTZ
    GROUP BY
        1
    ORDER BY
        1;
    """

    try:
        half_hourly_df_for_week = run_query(query)
        if not half_hourly_df_for_week.empty:
            all_half_hourly_data.append(half_hourly_df_for_week)
            print(f"   -> Success: Found {half_hourly_df_for_week.shape[0]} half-hourly records for the week.")
        else:
            print(f"   -> Info: No impression data found for this week.")

    except Exception as e:
        print(f"   -> ERROR processing week starting {current_week_start.date()}: {e}")

    # Move to the next week
    current_week_start = current_week_end
    progress_bar.update(1)

progress_bar.close()

# --- Final Processing and Display ---
if all_half_hourly_data:
    final_df = pd.concat(all_half_hourly_data, ignore_index=True)

    # Convert all numeric columns
    numeric_columns = [col for col in final_df.columns if col != 'ACTIVITY_HALF_HOUR']
    for col in numeric_columns:
        final_df[col] = pd.to_numeric(final_df[col])

    # --- SAVE TO PARQUET IN DATA FOLDER ---
    output_filename = f"data/half_hourly_impressions_{ANALYSIS_START_DATE}_to_{ANALYSIS_END_DATE}.parquet"
    final_df.to_parquet(output_filename, index=False)
    print(f"\n✅ Data successfully saved to {output_filename}")

    show_table(final_df.head(25), "Aggregated Half-Hourly Impression Metrics")

    print(f"\nDataset Summary:")
    print(f"Total records: {final_df.shape[0]:,}")
    print(f"Total columns: {final_df.shape[1]}")
    print(f"Memory usage: {final_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
else:
    print("\nNo impression data found for the entire specified period.")


--- Collecting Half-Hourly Raw Impression Metrics (Weekly Pulls) ---
Period: 2025-03-01 to 2025-09-30


Processing remaining weeks:   3%|▎         | 1/31 [12:12<6:06:24, 732.82s/it]

Processing week: 2025-03-01 to 2025-03-08...


   -> Info: No impression data found for this week.
Processing week: 2025-03-08 to 2025-03-15...


   -> Success: Found 48 half-hourly records for the week.
Processing week: 2025-03-15 to 2025-03-22...


KeyboardInterrupt: 

# auctions - users

In [ ]:

# --- CONFIGURATION ---
ANALYSIS_START_DATE = '2025-03-01'
ANALYSIS_END_DATE = '2025-09-30'

print(f"\n--- Collecting Half-Hourly Raw Auction User Metrics (Weekly Pulls) ---")
print(f"Period: {ANALYSIS_START_DATE} to {ANALYSIS_END_DATE}")

# --- Main Loop ---
all_half_hourly_data = []
current_week_start = datetime.strptime(ANALYSIS_START_DATE, '%Y-%m-%d')
end_date_obj = datetime.strptime(ANALYSIS_END_DATE, '%Y-%m-%d')

week_count = (end_date_obj - current_week_start).days // 7 + 1
progress_bar = tqdm(total=week_count, desc="Processing weeks")

while current_week_start < end_date_obj:
    current_week_end = current_week_start + timedelta(days=7)

    week_start_str = current_week_start.strftime('%Y-%m-%d 00:00:00')
    week_end_str = current_week_end.strftime('%Y-%m-%d 00:00:00')

    print(f"Processing week: {current_week_start.date()} to {current_week_end.date()}...")

    query = f"""
    SELECT
        -- Half-hourly time bucket
        DATEADD(MINUTE, FLOOR(EXTRACT(MINUTE FROM CREATED_AT)/30)*30, 
                DATE_TRUNC('HOUR', CREATED_AT))::TIMESTAMP_NTZ AS activity_half_hour,

        -- Auction metrics
        COUNT(AUCTION_ID) AS half_hourly_auction_count,
        COUNT(DISTINCT AUCTION_ID) AS half_hourly_unique_auctions,
        COUNT(DISTINCT OPAQUE_USER_ID) AS half_hourly_auction_users,
        
        -- User activity intensity
        COALESCE(AVG(user_auction_counts.auctions_per_user), 0) AS half_hourly_avg_auctions_per_user,
        COALESCE(MIN(user_auction_counts.auctions_per_user), 0) AS half_hourly_min_auctions_per_user,
        COALESCE(MAX(user_auction_counts.auctions_per_user), 0) AS half_hourly_max_auctions_per_user,
        COALESCE(STDDEV(user_auction_counts.auctions_per_user), 0) AS half_hourly_stddev_auctions_per_user
    FROM
        AUCTIONS_USERS au
    LEFT JOIN (
        SELECT 
            OPAQUE_USER_ID,
            DATEADD(MINUTE, FLOOR(EXTRACT(MINUTE FROM CREATED_AT)/30)*30, 
                    DATE_TRUNC('HOUR', CREATED_AT))::TIMESTAMP_NTZ AS half_hour_bucket,
            COUNT(AUCTION_ID) AS auctions_per_user
        FROM AUCTIONS_USERS
        WHERE CREATED_AT >= '{week_start_str}'::TIMESTAMP_NTZ
        AND CREATED_AT < '{week_end_str}'::TIMESTAMP_NTZ
        GROUP BY OPAQUE_USER_ID, half_hour_bucket
    ) user_auction_counts ON au.OPAQUE_USER_ID = user_auction_counts.OPAQUE_USER_ID 
                        AND DATEADD(MINUTE, FLOOR(EXTRACT(MINUTE FROM au.CREATED_AT)/30)*30, 
                                    DATE_TRUNC('HOUR', au.CREATED_AT))::TIMESTAMP_NTZ = user_auction_counts.half_hour_bucket
    WHERE
        au.CREATED_AT >= '{week_start_str}'::TIMESTAMP_NTZ
        AND au.CREATED_AT < '{week_end_str}'::TIMESTAMP_NTZ
    GROUP BY
        1
    ORDER BY
        1;
    """

    try:
        half_hourly_df_for_week = run_query(query)
        if not half_hourly_df_for_week.empty:
            all_half_hourly_data.append(half_hourly_df_for_week)
            print(f"   -> Success: Found {half_hourly_df_for_week.shape[0]} half-hourly records for the week.")
        else:
            print(f"   -> Info: No auction user data found for this week.")

    except Exception as e:
        print(f"   -> ERROR processing week starting {current_week_start.date()}: {e}")

    # Move to the next week
    current_week_start = current_week_end
    progress_bar.update(1)

progress_bar.close()

# --- Final Processing and Display ---
if all_half_hourly_data:
    final_df = pd.concat(all_half_hourly_data, ignore_index=True)

    # Convert all numeric columns
    numeric_columns = [col for col in final_df.columns if col != 'ACTIVITY_HALF_HOUR']
    for col in numeric_columns:
        final_df[col] = pd.to_numeric(final_df[col])

    # --- SAVE TO PARQUET IN DATA FOLDER ---
    output_filename = f"data/half_hourly_auction_users_{ANALYSIS_START_DATE}_to_{ANALYSIS_END_DATE}.parquet"
    final_df.to_parquet(output_filename, index=False)
    print(f"\n✅ Data successfully saved to {output_filename}")

    show_table(final_df.head(25), "Aggregated Half-Hourly Auction User Metrics")

    print(f"\nDataset Summary:")
    print(f"Total records: {final_df.shape[0]:,}")
    print(f"Total columns: {final_df.shape[1]}")
    print(f"Memory usage: {final_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
else:
    print("\nNo auction user data found for the entire specified period.")



--- Collecting Hourly Raw Auction Metrics (Daily Pulls) ---
Period: 2025-03-01 to 2025-09-30
Processing date: 2025-03-01...
   -> Info: No auction data found for this day.
Processing date: 2025-03-02...
   -> Info: No auction data found for this day.
Processing date: 2025-03-03...
   -> Info: No auction data found for this day.
Processing date: 2025-03-04...
   -> Info: No auction data found for this day.
Processing date: 2025-03-05...
   -> Info: No auction data found for this day.
Processing date: 2025-03-06...
   -> Info: No auction data found for this day.
Processing date: 2025-03-07...
   -> Info: No auction data found for this day.
Processing date: 2025-03-08...
   -> Info: No auction data found for this day.
Processing date: 2025-03-09...
   -> Info: No auction data found for this day.
Processing date: 2025-03-10...
   -> Info: No auction data found for this day.
Processing date: 2025-03-11...
   -> Info: No auction data found for this day.
Processing date: 2025-03-12...
   -> 